In [1]:
from deeppavlov.models.bidirectional_lms import elmo_bilm
from deeppavlov.models.tokenizers.lazy_tokenizer import LazyTokenizer
from nltk.tokenize.moses import MosesDetokenizer
import numpy as np
from typing import List
from scipy.stats import kurtosis
from scipy.stats.mstats import gmean
import re


import pandas as pd

[nltk_data] Downloading package punkt to /home/sultanov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sultanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/sultanov/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/sultanov/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [140]:
class ElmoAug:
    
    def __init__(self, model_dir="/cephfs/home/sultanov/elmo_lm/lib/python3.6/site-packages/download/bidirectional_lms/elmo_en_news"):
        self.tokenizer = LazyTokenizer()
        self.elmo_lm   = elmo_bilm.ELMoEmbedder(model_dir=model_dir)
        self.elmo_vocab = np.array(self.elmo_lm.get_vocab())
        self.detokenizer = MosesDetokenizer()
        self.replacement_patterns = [
             (r'won\'t', 'will not'),
             (r'can\'t', 'cannot'),
             (r'i\'m', 'i am'),
             (r'ain\'t', 'is not'),
             (r'(\w+)\'ll', '\g<1> will'),
             (r'(\w+)n\'t', '\g<1> not'),
             (r'(\w+)\'ve', '\g<1> have'),
             (r'(\w+)\'s', '\g<1> is'),
             (r'(\w+)\'re', '\g<1> are'),
             (r'(\w+)\'d', '\g<1> would')
        ]
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
        
    def _preproccesing(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        return s
    
    def _weighted_sum_distr_by_posistion_in_sent(self, distr):
        if len(distr) > 4:
            weights = np.array([0.2, 0.35] + [0.5]*(len(distr) - 4) + [0.65, 0.8])
        else:
            weights = 0.5*np.ones(len(distr))
        left = distr[:,0,:]
        right = distr[:,1,:]
        right = right.transpose([1, 0]) * (1-weights)
        right = right.transpose([1, 0])
        left = left.transpose([1, 0]) * weights
        left = left.transpose([1, 0])
        return right + left
    
    def _get_index_in_vocab(self, token):
        indx = np.where(self.elmo_vocab == token)[0]
        return indx[0] if indx.size > 0 else None

    def _blend_dist(self, batch_distr, num_method):
        """
        blending distr from left and right context
        method 0:
            sum two distr along left right context
        method 1:
            weighted sum by place of word in sentence
        """
        if num_method == 0:
            return [np.sum(distr, axis=1) for distr in batch_distr]
        
        elif num_method == 1:
            return [self._weighted_sum_distr_by_posistion_in_sent(distr) for distr in batch_distr]
        
        elif num_method == 2:
            return [np.min(distr, axis=1) for distr in batch_distr]
        
        elif num_method == 3:
            return [gmean(distr, axis=1) for distr in batch_distr]
    
    def _softmax(self, x):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()
    
    def _sent_aug(self, source_sentence: List[str], distr: np.ndarray, threshold: float=0.3, replace_freq: float=0.5):
        """Sampling words from the distribution"""
        result = []
        for i, token in enumerate(source_sentence):
            if distr.mask[i].all():
                #source token
                result.append(token)
            else:
                #replace
                idx_word = np.random.choice(len(distr[i]), replace=False, p=self._softmax(distr[i]).filled(0))
                result.append(self.elmo_vocab[idx_word])
    
    def _get_threshold_masked(self, data, indx_source_token, threshold, replace_freq):
        """
        Creating mask:
            source word - deleted
            word with probability less that threshold - deleted
            amount of words that will be replaced ≈ replace_freq * len(sentence)
            if amount of words with probability > threshold <= replace_freq * len(sentence)
            then will printed warning and all words with probability > threshold will be replaced
        """
        #Creating a mask based on threshold
        assert(len(data) != len(indx_source_token))
        mask = data > threshold
        #Creating a mask that marks the source tokens, and merging it with previous mask
        onehot_indx = np.zeros((len(indx_source_token), len(el.elmo_vocab)))
        onehot_indx[np.arange(len(indx_source_token)), indx_source_token] = 1
        mask = (mask + onehot_indx).astype(bool)
        #Creating a mask based on replace frequence, and merging it with previous
        word_mask = mask.any(axis=1)
        if replace_freq >= len(word_mask.nonzero()[0])/len(word_mask):
            print('warning')
            word_mask = word_mask
        else:
            freq = (replace_freq * len(word_mask)) / len(word_mask.nonzero()[0])
            word_mask = (np.random.binomial(1, p=freq, size=(len(word_mask))) * word_mask).astype(bool)
        mask = (mask.T*word_mask).T
        return np.ma.masked_array(data=data, mask=~mask, fill_value=np.nan)
        
    def _batch_sent(self, batch_sent: List[str], threshold: float, replace_freq: float, num_method_blend: int) -> List[str]:
        """
        Replaces some words in the original sentence with words from the language model with frequency p
        Args:
            batch_token: Sentences to be augmented 
            threshold: words with probability < threshold will not be considerated like replacement
            p: frequency of replacing words
            num_method_blend: method of merging two distributions: left-context and right-context
        Returns: 
            Contains the augmented sentences
        """
        batch_sent_prep         = [self._preproccesing(sent) for sent in batch_sent]
        batch_token             = self.tokenizer(batch_sent_prep)
        batch_indx_source_token = [np.array(list(map(self._get_index_in_vocab, sent))) for sent in batch_token]
        batch_distr             = self._blend_dist(self.elmo_lm(batch_token), num_method_blend)
        batch_mask_distr        = [self._get_threshold_masked(batch_distr[i], batch_indx_source_token[i] threshold, replace_freq) for i in range(len(batch_token))]
        batch_aug_token         = [self._sent_aug(batch_token[i], batch_mask_distr[i]) for i in range(len(batch_token))]
        return batch_aug_token        
    
    def __call__(self, batch_sent: List[str], threshold: float, replace_freq: float, num_method_blend: int):
        batch_aug_token = self._batch_sent(batch_sent, threshold, replace_freq, num_method_blend)
        return [self.detokenizer.detokenize(i, return_str=True) for i in batch_aug_token]

In [141]:
el = ElmoAug()

****************************************************************************************************
/cephfs/home/sultanov/elmo_lm/lib/python3.6/site-packages/download/bidirectional_lms/elmo_en_news
USING SKIP CONNECTIONS
INFO:tensorflow:Restoring parameters from /cephfs/home/sultanov/elmo_lm/lib/python3.6/site-packages/download/bidirectional_lms/elmo_en_news/model.ckpt-935588


2018-10-31 15:23:51.531 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from /cephfs/home/sultanov/elmo_lm/lib/python3.6/site-packages/download/bidirectional_lms/elmo_en_news/model.ckpt-935588


In [142]:
test_sentences = \
["Almost half of all iPhone owners have broken their screens, not just once but an average of two times each.",\
   "i really don't understand your point.\xa0 It seems that you are mixing apples and oranges.",\
   "shut the fuck up. you and the rest of your faggot friends should be burned at the stake",\
   "That you are an idiot who understands neither taxation nor women's health.",\
   "What on Earth is that about? Is it what's going to get him fired eventually?",\
   "This is a doctrine of constitutional interpretation that says that a constitution is organic and must be read in a broad and liberal manner so as to adapt it to changing times.",\
   "In the 2000s, music notation typically means the written expression of music notes and rhythms on paper using symbols.",\
   "Most of the mathematical notation in use today was not invented until the 16th century.[52] Before that, mathematics was written out in words, limiting mathematical discovery.",\
   "Physical geography deals with the study of processes and patterns in the natural environment like the atmosphere, hydrosphere, biosphere, and geosphere.",\
   "An autobiography is written by the person himself or herself, sometimes with the assistance of a collaborator or ghostwriter.",\
    "You fuck your dad.",\
    "Yeah and where are you now?",\
    "shut the fuck up. you and the rest of your faggot friends should be burned at the stake",\
    "you are a land creature. You would drown....",\
    "But how would you actually get the key out?",\
    "fucking behave then you prick!",\
    "You right if you are relaxe then you can give better result or perform and your identity should be from your work.",\
    "The laughs you two heard were triggered by memories of his own high-flying exits off moving beasts",\
 "Well, you guys have gone and done it now. You put the words 'China' and 'Chinese' up the required number of times for the dating Asians ad to come up. Evidently, Ms. Zhang, 50Kg and 168cm [for a BMI of 17.8] from 'HuNan China' wants to meet me. She has her little mouth open like she's speaking. What's that you ask, Zhang? Well, yes, as a matter of fact I am a physician.  Why are you clapping your hands together and jumping up and down?  Stop that squealing, young lady and 'exprain' yourself!",\
 "Fact : Georgia passed a strict immigration policy and most of the Latino farm workers left the area. Vidalia Georgia now has over 3000 agriculture job openings and they have been able to fill about 250 of them in past year. All you White Real Americans who are looking for work that the Latinos stole from you..Where are you ? The jobs are i Vadalia just waiting for you..Or maybe its the fact that you would rather collect unemployment like the rest of the Tea Klaners.. You scream..you complain..and you sit at home in your wife beaters and drink beer..Typical Real White Tea Klan..."
]
len(test_sentences)

20

In [143]:
%%time
#for method in range(3):
el([test_sentences[0]], 0.2, 0.5, 3)

[array([5512,  347,    7,   70, 2914, 1691,   27, 1960,   40, 6124,    4,
         34,  100,  480,   44,   32,  596,    7,   67,  400,  290,    5])]
warning


TypeError: _sent_aug() missing 1 required positional argument: 'threshold'

In [21]:
import numpy as np

In [163]:
data = np.array([[0.3, 0.9, 0.1, 0.15, 0.3],\
               [0.1, 0.1, 0.8, 0.3, 0.3],\
               [0.1, 0.1, 0.1, 0.1, 0.1],\
               [0.1, 0.1, 0.1, 0.1, 0.1],\
               [0.3, 0.4, 0.3, 0.15, 0.3],\
               [0.1, 0.1, 0.8, 0.3, 0.3],\
               [0.1, 0.1, 0.1, 0.1, 0.1],\
               [0.3, 0.4, 0.3, 0.15, 0.3],\
               [0.1, 0.1, 0.8, 0.3, 0.3],\
               [0.1, 0.1, 0.1, 0.1, 0.1]])
threshold = 0.25
replace_freq = 0.39
mask = data > threshold
word_mask = mask.any(axis=1)
np.ma.masked_array(data=data, mask=~mask, fill_value=np.nan)
mask

array([[ True,  True, False, False,  True],
       [False, False,  True,  True,  True],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [ True,  True,  True, False,  True],
       [False, False,  True,  True,  True],
       [False, False, False, False, False],
       [ True,  True,  True, False,  True],
       [False, False,  True,  True,  True],
       [False, False, False, False, False]])

In [36]:
def _threshold_mask(data, threshold, replace_freq):
    mask = data > threshold
    word_mask = mask.any(axis=1)
    if replace_freq >= len(word_mask.nonzero()[0])/len(word_mask):
        print('warning')
        word_mask = word_mask
    else:
        freq = (replace_freq * len(word_mask)) / len(word_mask.nonzero()[0])
        word_mask = (np.random.binomial(1, p=freq, size=(len(word_mask))) * word_mask).astype(bool)
    mask = (mask.T*word_mask).T
    return np.ma.masked_array(data=data, mask=~mask, fill_value=np.nan)

In [52]:
a = _threshold_mask(data, threshold, replace_freq)
kl = el._softmax(a[0])
kl

masked_array(data=[0.2616349863059801, 0.4767300273880397, --, --,
                   0.2616349863059801],
             mask=[False, False,  True,  True, False],
       fill_value=nan)

In [76]:
np.sum(kl)
kl /= np.sum(kl)
kl.set_fill_value(0)
kl.filled(0)
kl

masked_array(data=[0.2616349863059801, 0.4767300273880397, --, --,
                   0.2616349863059801],
             mask=[False, False,  True,  True, False],
       fill_value=0.0)

In [95]:
kl.mask[0] = True
kl

masked_array(data=[--, 0.4767300273880397, --, --, 0.2616349863059801],
             mask=[ True, False,  True,  True, False],
       fill_value=0.0)

In [86]:
np.random.choice(np.arange(len(a[0])), replace=False, p=kl.filled(0), size=3)

array([0, 1, 4])

In [38]:
len(a[0])

5

In [139]:
def get_index_in_vocab(token):
    indx = np.where(np.array(el.elmo_lm.get_vocab()) == token)[0]
    return indx[0] if indx.size > 0 else None
np.array(list(map(get_index_in_vocab, ['<\S>','<S>','<UNK>','the',','])))

array(<map object at 0x7fdef0737ef0>, dtype=object)

In [119]:
el.elmo_lm.get_vocab()

['</S>',
 '<S>',
 '<UNK>',
 'the',
 ',',
 '.',
 'to',
 'of',
 'and',
 'a',
 'in',
 '"',
 "'s",
 'that',
 'for',
 'on',
 'is',
 'The',
 'was',
 'with',
 'said',
 'as',
 'at',
 'it',
 'by',
 'from',
 'be',
 'have',
 'he',
 'has',
 'his',
 'are',
 'an',
 ')',
 'not',
 '(',
 'will',
 'who',
 'I',
 'had',
 'their',
 '--',
 'were',
 'they',
 'but',
 'been',
 'this',
 'which',
 'more',
 'or',
 'its',
 'would',
 'about',
 ':',
 'after',
 'up',
 '$',
 'one',
 'than',
 'also',
 "'t",
 'out',
 'her',
 'you',
 'year',
 'when',
 'It',
 'two',
 'people',
 '-',
 'all',
 'can',
 'over',
 'last',
 'first',
 'But',
 'into',
 "'",
 'He',
 'A',
 'we',
 'In',
 'she',
 'other',
 'new',
 'years',
 'could',
 'there',
 '?',
 'time',
 'some',
 'them',
 'if',
 'no',
 'percent',
 'so',
 'what',
 'only',
 'government',
 'million',
 'just',
 'U.S.',
 'him',
 'before',
 'most',
 'like',
 'because',
 'now',
 'three',
 ';',
 'being',
 'against',
 'do',
 'Obama',
 'where',
 'made',
 'Mr',
 'many',
 'New',
 'back',
 'an

In [144]:
np.eye(len(el.elmo_lm.get_vocab()))[np.array([0, 3, 2])]

MemoryError: 

In [159]:
a = np.array([1, 0, 2])
b = np.zeros((len(a), 3))
b[np.arange(len(a)), a] = 1
b.astype(bool)

array([[False,  True, False],
       [ True, False, False],
       [False, False,  True]])

In [160]:
yy = np.array([[0,0,0],[0,1,0],[0,0,0]])
yy

array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 0]])

In [161]:
(b + yy).astype(bool)

array([[False,  True, False],
       [ True,  True, False],
       [False, False,  True]])

In [162]:
el._softmax()

array([0.26894142, 0.73105858])